In [1]:
import pandas as pd
import numpy as np
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import os
import folium
import vincent,json
import csv
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

def data_input():
    filename = input("Please input the dataset name: ")
    name = input("Please input the specific column name of 'Name' of your dataset:")
    date = input("Please input the specific column name of 'Date' of your dataset:")
    lon = input("Please input the specific column name of 'Longitute' of your dataset:")
    lat = input("Please input the specific column name of 'Latitue' of your dataset:")
    category = input("Please input the specific column name of 'Category' of your dataset:")
    num1 = input("Please input the specific first column name of 'Numeric' of your dataset:")
    num2 = input("Please input the specific first column name of 'Numeric' of your dataset:")
    num3 = input("Please input the specific first column name of 'Numeric' of your dataset:")
    
    #load the data
    df=read_csv(filename)
    
    #make the folium map
    map=folium.Map(location=[df[lat].mean(),df[lon].mean()],zoom_start=6,tiles='stamenterrain')
    
    #make the vincent chart for category 
    df_group=df.groupby(cat)
    df_group_sum=df_group[num1,num2].sum()
    group=vincent.GroupedBar(df_group_sum,width=350,height=200)
    group.legend(title='Numeric data')
    group.colors(brew='Pastel1')
    group.axis_titles(x='Categories', y='Sum of the Quant')

    #make the vincent line chart for datetime
    df_group2=df.groupby(date)
    df_group_sum2=df_group2[num1,num2].sum()
    lines = vincent.Line(df_group_sum2,width=350,height=200)
    lines.scales[0].type = 'ordinal'
    lines.axis_titles(x='Datetime', y='Sum of the Quant')
    lines.legend(title='Categories')
    
    #add quant1 on the map
    fg=folium.FeatureGroup(name=num1)
    for lat,lon,cat,num,num_2,name in zip(df[lat],df[lon],df[category],df[num1],df[num2],df[name]):
    #make the html click popup
        html="""
    name: {}<br>
    date: {}<br>
    num1: {}<br>
    num2: {}<br>
    num3: {}<br>
    category: {}<br>
        """
        html = html.format(name,num,num_2,cat)
        test = folium.IFrame(html, width=150, height=100)
        fg.add_child(folium.CircleMarker(location=[lat,lon],popup=(folium.Popup(test)),radius=num/10,color=Color(cat),fill_color=Color(cat)))
    map.add_child(fg)
    
    
    
    #add quant2 on the map
    fh=folium.FeatureGroup(name="Numeric2 Data,Categories Chart")
    for lat,lon,name,num2 in zip(df[lat],df[lon],df[category],df[num2]):
    #make the groupbar chart click popup
        popup= folium.Popup(max_width=800)
        folium.Vega(group, height=250, width=450).add_to(popup)
        fh.add_child(folium.RegularPolygonMarker(location=[lat,lon],popup=popup,radius=num2/10,color=Color(name),fill_color=Color(name),number_of_sides=3))
    map.add_child(fh)
    
    
    #add quant3 on the map
    fk=folium.FeatureGroup(name="Time,Datetime Chart")
    for lat,lon,name,time in zip(df[lat],df[lon],df[category],df[date]):
    #make the line chart click popup
        popup = folium.Popup(max_width=800)
        folium.Vega(lines, height=250, width=450).add_to(popup)
        fk.add_child(folium.RegularPolygonMarker(location=[lat,lon],popup=popup,radius=num3/10,color=Color(name),fill_color=Color(name),number_of_sides=4))
    map.add_child(fk)
    
   
    #add layer contraller
    map.add_child(folium.LayerControl()

    #save the map as html file
    map.save(outfile='part3map_3.0.html')